In [1]:
from glob import glob
import anndata
import pandas as pd
from scipy import io
from scipy import sparse
from tqdm import tqdm
import anndata


In [2]:
# create a dictionary with the paths for the subsampling folder where snakemake outputs are for each dataset

datasets = ['heart10k', 'pbmc10k', 'neurons10k']

subsampling_paths = {
'heart10k' :'./heart_10k_v3/genecounts/',    
'pbmc10k' :'./pbmc_10k_v3/genecounts/',    
'neurons10k' :'./neuron_10k_v3/genecounts/',    
}

# these are annotation files downloaded from the 10x website, they are used to parse the barcodes and total number of cells in each dataset
cluster_files = {
'heart10k' :'./heart10k_clusters.csv',    
'pbmc10k' :'./pbmc10k_clusters.csv',    
'neurons10k' :'./neurons10k_clusters.csv',     
    
}

In [3]:
# check that output folders eist
for ds in datasets:
    print('\n files for dataset', ds, ':')
    !ls -v {subsampling_paths[ds]}


 files for dataset heart10k :
genecounts_subsampled_0        genecounts_subsampled_6400000
genecounts_subsampled_100000   genecounts_subsampled_9050967
genecounts_subsampled_141421   genecounts_subsampled_12800000
genecounts_subsampled_200000   genecounts_subsampled_18101934
genecounts_subsampled_282843   genecounts_subsampled_25600000
genecounts_subsampled_400000   genecounts_subsampled_36203867
genecounts_subsampled_565685   genecounts_subsampled_51200000
genecounts_subsampled_800000   genecounts_subsampled_72407734
genecounts_subsampled_1131371  genecounts_subsampled_102400000
genecounts_subsampled_1600000  genecounts_subsampled_144815469
genecounts_subsampled_2262742  genecounts_subsampled_204800000
genecounts_subsampled_3200000  genecounts_subsampled_289630938
genecounts_subsampled_4525483

 files for dataset pbmc10k :
genecounts_subsampled_0        genecounts_subsampled_9050967
genecounts_subsampled_100000   genecounts_subsampled_12800000
genecounts_subsampled_141421   genecount

In [4]:
# check one folder
!ls -lah {subsampling_paths['pbmc10k']}/genecounts_subsampled_100000/

total 1.5M
drwxrwxr-x.  3 munfred munfred 4.0K Sep  7 18:17 .
drwxrwxr-x. 29 munfred munfred 4.0K Sep  7 20:17 ..
drwxrwxr-x.  2 munfred munfred 4.0K Sep  7 18:17 genecounts
-rw-rw-r--.  1 munfred munfred 223K Sep  7 18:17 genecounts.barcodes.txt
-rw-rw-r--.  1 munfred munfred 640K Sep  7 18:17 genecounts.genes.txt
-rw-rw-r--.  1 munfred munfred 557K Sep  7 18:17 genecounts.mtx


In [5]:
for ds in datasets:
    print(' Now processing dataset: ', ds)
    annotation = pd.read_csv(cluster_files[ds], index_col=0)
    annotation.index = annotation.index.str.split('-').str.get(0)
    display(annotation.head(2))


    genes = pd.read_csv(subsampling_paths[ds] + 'genecounts_subsampled_0/genecounts.genes.txt', names=['gene_id']).set_index('gene_id')
    display(genes.head(2))

    base_ad = anndata.AnnData(obs=annotation, var=genes)

    for mtx_file in tqdm(glob( subsampling_paths[ds] + '/genecounts_subsampled_*/genecounts.mtx')):
        ss_depth = mtx_file.split('ed_')[-1].split('/')[0]
        X = io.mmread(mtx_file).tocsr()

        barcodes = pd.read_csv(
            mtx_file.replace('.mtx', '.barcodes.txt'),
            names=['barcode'],
            index_col=0
        )
        curr_adata = anndata.AnnData(X=X, obs=barcodes, var=genes)

        shared_barcodes = curr_adata.obs_names.intersection(base_ad.obs_names)
        base_idx = base_ad.obs_names.get_indexer(shared_barcodes)
        curr_idx = curr_adata.obs_names.get_indexer(shared_barcodes)

        idxmtx = sparse.eye(base_ad.shape[0], format="csr")[:, base_idx]
        # For some reason this is way faster than equivalent indexing statement
        base_ad.layers[ss_depth] = idxmtx @ curr_adata.X[curr_idx]
        


    # This is a check to ensure the anndata layers are correct
    for depth in base_ad.layers.keys():
        print('UMIs: \t', int(base_ad.layers[depth].sum()),'\t Reads: \t', depth )
    print(base_ad)


    base_ad.write( ds + '_subsamples.h5ad')


 Now processing dataset:  heart10k


,Cluster
Barcode,
AAACCCAAGCGAGTCA,2
AAACCCAAGGTCATTC,1


""
gene_id
ENSMUSG00000051951.5
ENSMUSG00000102851.1


100%|██████████| 25/25 [03:44<00:00,  8.44s/it]


UMIs: 	 19373016 	 Reads: 	 51200000
UMIs: 	 1341886 	 Reads: 	 3200000
UMIs: 	 950966 	 Reads: 	 2262742
UMIs: 	 72563226 	 Reads: 	 289630938
UMIs: 	 59635 	 Reads: 	 141421
UMIs: 	 72679923 	 Reads: 	 0
UMIs: 	 1891433 	 Reads: 	 4525483
UMIs: 	 3747318 	 Reads: 	 9050967
UMIs: 	 337292 	 Reads: 	 800000
UMIs: 	 5256131 	 Reads: 	 12800000
UMIs: 	 673164 	 Reads: 	 1600000
UMIs: 	 45811720 	 Reads: 	 144815469
UMIs: 	 238550 	 Reads: 	 565685
UMIs: 	 84409 	 Reads: 	 200000
UMIs: 	 2664470 	 Reads: 	 6400000
UMIs: 	 476734 	 Reads: 	 1131371
UMIs: 	 35021220 	 Reads: 	 102400000
UMIs: 	 42202 	 Reads: 	 100000
UMIs: 	 7346236 	 Reads: 	 18101934
UMIs: 	 58484104 	 Reads: 	 204800000
UMIs: 	 168862 	 Reads: 	 400000
UMIs: 	 10223782 	 Reads: 	 25600000
UMIs: 	 14134634 	 Reads: 	 36203867
UMIs: 	 26246409 	 Reads: 	 72407734
UMIs: 	 119301 	 Reads: 	 282843
AnnData object with n_obs × n_vars = 7713 × 36047 
    obs: 'Cluster'
    layers: '51200000', '3200000', '2262742', '289630938',

...

UMIs: 	 19226439 	 Reads: 	 51200000
UMIs: 	 1353775 	 Reads: 	 3200000
UMIs: 	 80563999 	 Reads: 	 409600000
UMIs: 	 959447 	 Reads: 	 2262742
UMIs: 	 68096948 	 Reads: 	 289630938
UMIs: 	 59981 	 Reads: 	 141421
UMIs: 	 95614832 	 Reads: 	 0
UMIs: 	 1907442 	 Reads: 	 4525483
UMIs: 	 3773430 	 Reads: 	 9050967
UMIs: 	 340898 	 Reads: 	 800000
UMIs: 	 5284136 	 Reads: 	 12800000
UMIs: 	 680142 	 Reads: 	 1600000
UMIs: 	 44258659 	 Reads: 	 144815469
UMIs: 	 241179 	 Reads: 	 565685
UMIs: 	 85048 	 Reads: 	 200000
UMIs: 	 2685602 	 Reads: 	 6400000
UMIs: 	 481377 	 Reads: 	 1131371
UMIs: 	 34214360 	 Reads: 	 102400000
UMIs: 	 42464 	 Reads: 	 100000
UMIs: 	 92430124 	 Reads: 	 579261875
UMIs: 	 7372370 	 Reads: 	 18101934
UMIs: 	 55739226 	 Reads: 	 204800000
UMIs: 	 170422 	 Reads: 	 400000
UMIs: 	 10231232 	 Reads: 	 25600000
UMIs: 	 14094876 	 Reads: 	 36203867
UMIs: 	 25870880 	 Reads: 	 72407734
UMIs: 	 120614 	 Reads: 	 282843
AnnData object with n_obs × n_vars = 11769 × 35606 


,Cluster
Barcode,
AAACCCAAGCAACTCT,5
AAACCCACACGCGGTT,7


""
gene_id
ENSMUSG00000051951.5
ENSMUSG00000102851.1


100%|██████████| 25/25 [05:05<00:00, 11.70s/it]


UMIs: 	 20571740 	 Reads: 	 51200000
UMIs: 	 1433742 	 Reads: 	 3200000
UMIs: 	 1015726 	 Reads: 	 2262742
UMIs: 	 74982204 	 Reads: 	 289630938
UMIs: 	 63740 	 Reads: 	 141421
UMIs: 	 83639178 	 Reads: 	 0
UMIs: 	 2022292 	 Reads: 	 4525483
UMIs: 	 4003347 	 Reads: 	 9050967
UMIs: 	 360887 	 Reads: 	 800000
UMIs: 	 5611289 	 Reads: 	 12800000
UMIs: 	 719857 	 Reads: 	 1600000
UMIs: 	 48061681 	 Reads: 	 144815469
UMIs: 	 254964 	 Reads: 	 565685
UMIs: 	 90110 	 Reads: 	 200000
UMIs: 	 2847599 	 Reads: 	 6400000
UMIs: 	 509956 	 Reads: 	 1131371
UMIs: 	 36934330 	 Reads: 	 102400000
UMIs: 	 45010 	 Reads: 	 100000
UMIs: 	 7838130 	 Reads: 	 18101934
UMIs: 	 60946675 	 Reads: 	 204800000
UMIs: 	 180489 	 Reads: 	 400000
UMIs: 	 10892676 	 Reads: 	 25600000
UMIs: 	 15038633 	 Reads: 	 36203867
UMIs: 	 27793736 	 Reads: 	 72407734
UMIs: 	 127561 	 Reads: 	 282843
AnnData object with n_obs × n_vars = 11843 × 36047 
    obs: 'Cluster'
    layers: '51200000', '3200000', '2262742', '289630938

In [6]:
ls -lah

total 9.4G
drwxrwxr-x.  8 munfred munfred 4.0K Sep  8 20:34 ./
drwxrwxr-x. 13 munfred munfred 4.0K Sep  7 16:58 ../
-rw-rw-r--.  1 munfred munfred 7.7K Sep  7 17:04 1) cell-depth-tradeoff-subsampling.py
-rw-rw-r--.  1 munfred munfred  20K Sep  8 20:33 2) create_H5AD.ipynb
-rw-rw-r--.  1 munfred munfred 6.8K Sep  7 20:56 3) run_scvi.py
-rw-rw-r--.  1 munfred munfred 8.0M Sep  7 17:46 4) make_all_plots.ipynb
drwxrwxr-x.  5 munfred munfred 4.0K Sep  7 17:38 benchmarks/
-rw-rw-r--.  1 munfred munfred 2.4K Sep  7 17:01 cell-depth-tradeoff-metadata.tsv
-rw-rw-r--.  1 munfred munfred 160K Sep  8 20:03 heart10k_clusters.csv
-rw-rw-r--.  1 munfred munfred 1.6G Sep  8 20:24 heart10k_subamples.h5ad
drwxrwxr-x.  5 munfred munfred 4.0K Sep  7 18:07 heart_10k_v3/
drwxrwxr-x.  2 munfred munfred 4.0K Sep  7 20:38 .ipynb_checkpoints/
drwxrwxr-x.  5 munfred munfred 4.0K Sep  7 18:06 neuron_10k_v3/
-rw-rw-r--.  1 munfred munfred 246K Sep  8 20:03 neurons10k_clusters.csv
-rw-rw-r--.  1 munfred munfred 2.3